In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [2]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [3]:
import boto3
from decimal import Decimal
# Get the service resource.
dynamodb = boto3.resource('dynamodb', region_name='ap-northeast-2')
table = dynamodb.Table('dsnia_imagelabelsV2')
# table = dynamodb.Table('dsnia_imagelabels')

In [4]:
course_list = ['청도','포항','중문','시흥','거창','속리산','춘천']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
  '시흥':'MGC004',
  '거창':'MGC005',
  '속리산':'MGC006',
  '춘천':'MGC007'
}

folder_type = ['rgb','ndvi','thumb']

In [5]:
AWS_folder = os.getcwd()

b_folder = os.path.join(AWS_folder,'BACKUPS')
b_folder

src_folder = os.path.join(AWS_folder,'dataDownloaded')
src_folder

out_folder = os.path.join(AWS_folder,'filepathBackUp')
out_folder

'c:\\Users\\yc463\\Documents\\JamesLocal\\DS\\Drone\\dsmapv2\\src\\AWS\\filepathBackUp'

In [6]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError



recentDB = glob.glob(os.path.join(b_folder ,'Total*.pickle'), recursive=True)[-1]
df = pd.read_pickle(recentDB)

alldata = df.to_dict('records')

In [7]:
len(alldata)

184082

In [8]:
[x for x in alldata if 'DJI_20230910164012' in x['json']['originalFileJPG']]

[{'metadata': 'MGC00320230910해안 6H물리적피해,장비,스켈핑_2',
  'date': '20230910',
  'area': '해안 6H',
  'courseid': 'MGC003',
  'json': {'originalFileGrp': ['ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/해안 6H/2023_0910_중문골프클럽_해안6H_1.5m/20230910_1637-1651_물리적피해,장비,스켈핑/DJI_20230910164012_0073_MS_G.TIF',
    'ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/해안 6H/2023_0910_중문골프클럽_해안6H_1.5m/20230910_1637-1651_물리적피해,장비,스켈핑/DJI_20230910164012_0073_MS_NIR.TIF',
    'ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/해안 6H/2023_0910_중문골프클럽_해안6H_1.5m/20230910_1637-1651_물리적피해,장비,스켈핑/DJI_20230910164012_0073_MS_RE.TIF',
    'ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/해안 6H/2023_0910_중문골프클럽_해안6H_1.5m/20230910_1637-1651_물리적피해,장비,스켈핑/DJI_20230910164012_0073_MS_R.TIF',
    'ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/해안 6H/2023_0910_중문골프클럽_해안6H_1.5m/20230910_1637-1651_물리적피해,장비,스켈핑/DJI_20230910164753_0073_MS_G.TIF',
    'ta/nas/10_원시데이터(3사_분류대상)/2023_0910_중문골프클럽_데이터 1차 분류_완/

In [9]:
def update_root(x):
  path = os.path.normpath(x)
  info = path.split(os.sep)
  index_ = [idx for idx, s in enumerate(info) if '2023' and '완' in s][0]
  return 'original/'+ ('/').join(info[index_:])

In [10]:
for jsonSinlgle in alldata:
  jsonSinlgle['json']['originalFileJPG'] = update_root(jsonSinlgle['json']['originalFileJPG'])
  jsonSinlgle['json']['originalFileGrp'] = [update_root(x) for x in jsonSinlgle['json']['originalFileGrp']]


In [12]:
alldata[10000]

{'metadata': 'MGC00520230906거창 클럽디 EAST 3이종잔디,라이그래스_1',
 'date': '20230906',
 'area': '거창 클럽디 EAST 3',
 'courseid': 'MGC005',
 'json': {'originalFileGrp': ['original/2023_0906_거창클럽디_맑은날_구름조금_데이터 1차 분류_완/거창 클럽디 EAST 3/2023_0906_거창클럽디_EAST3H_1.5m/20230906_1210-1235_이종잔디,라이그래스/DJI_20230906121230_0411_MS_G.TIF',
   'original/2023_0906_거창클럽디_맑은날_구름조금_데이터 1차 분류_완/거창 클럽디 EAST 3/2023_0906_거창클럽디_EAST3H_1.5m/20230906_1210-1235_이종잔디,라이그래스/DJI_20230906121230_0411_MS_NIR.TIF',
   'original/2023_0906_거창클럽디_맑은날_구름조금_데이터 1차 분류_완/거창 클럽디 EAST 3/2023_0906_거창클럽디_EAST3H_1.5m/20230906_1210-1235_이종잔디,라이그래스/DJI_20230906121230_0411_MS_R.TIF',
   'original/2023_0906_거창클럽디_맑은날_구름조금_데이터 1차 분류_완/거창 클럽디 EAST 3/2023_0906_거창클럽디_EAST3H_1.5m/20230906_1210-1235_이종잔디,라이그래스/DJI_20230906121230_0411_MS_RE.TIF'],
  'annotation': [],
  'labelBy': '',
  'GeoTagInfo': {'gps_altitude': Decimal('686.771'),
   'datetime_original': '2023:09:06 12:12:30',
   'coords': [Decimal('127.91078325000001'), Decimal('35.59604686111111')],
  

##### FINAL UPLOAD

In [13]:
with table.batch_writer(overwrite_by_pkeys=['metadata', 'id']) as batch:
  for index_, row_ in enumerate(alldata):
    print(index_, row_['id'], end='\r')
    batch.put_item(
      Item= row_
    )